In [45]:
#模拟优矿。
#1.0版本：。
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline #魔法命令，用于在jupyter 环境中正常显示图表
plt.rcParams['font.sans-serif']=['SimHei'] #用来在图中正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来在图中正常显示负号

datename = '221111'
convdataname = '{}{}{}'.format('convdata_',datename,'.csv')
initData=pd.read_csv(convdataname,index_col=0) #读取CSV数据文件。
#initData   #83833数据行。

In [46]:
TDate=initData['tradeDate'].drop_duplicates()
#TDate

In [47]:
#initData.query('secShortNameBond == "广汽转债"')

In [48]:
Start_Cash= 1000000
MyCash = 1000000   #现金。统计量，后续不断更新～～～～～～～～
HighValue = 0 #最高市值。统计量，后续不断更新～～～～～～～～
Withdraw = 0       #最大回撤。统计量，后续不断更新～～～～～～～～
TValue = 0         #总持仓市值。统计量，后续不断更新～～～～～～～～
HoldNum = 10       #持债支数。
MyPosition = {}    #持仓
Cost = {}          #成本
Value = {}         #市值
Profit = {}        #盈利

In [49]:
for today_date in TDate:
    data = initData.query('tradeDate=="' + today_date +'" and Daoqi < -365 ')
    data.set_index('secShortNameBond',inplace=True)
    #data['DoubleLow'] = data['closePriceBond'] + data['bondPremRatio']      #每天重新计算双低排名
    data['TriLow'] = data['closePriceBond'] + data['bondPremRatio'] + data['remainSize']     #每天重新计算双低排名
    #data = data.sort_values(by="closePriceBond" , ascending=True)          #按价格排序
    #data = data.sort_values(by="DoubleLow" , ascending=True)          #按双低排序
    data = data.sort_values(by="TriLow" , ascending=True)          #按三低排序
    Tvalue = 0
    for stock in list(MyPosition.keys()): #首先查看持仓盈利情况，先卖后买。通过list遍历，后续删除字典，不会报错！
        #CurPrice = data.query('secShortNameBond==@stock').closePriceBond[0] #获取持仓价格
        CurPrice = data.loc[stock,'closePriceBond'] #获取持仓价格
        Value[stock] = MyPosition[stock] * CurPrice * 10                    #更新单只市值
        Tvalue += Value[stock]                                              #更新总持仓市值
        Profit[stock] = Value[stock]/Cost[stock]-1                          #更新单只市值
        if Profit[stock] > 0.3:                                        #若盈利20%，则平仓。
            MyCash += Value[stock] * (1-0.0001)                        #卖出处理：现金增加，交易摩擦成本按万分之一。统计量更新～～～
            Tvalue -= Value[stock]                                     #卖出处理：总持仓市值减少。                  统计量更新～～～
            MyPosition[stock] = 0                                     #卖出处理：持仓数量、成本、市值同步
            del MyPosition[stock]
            print('{} 卖出 {},价格：{},金额：{},持仓市值{} ,现金{} ,'.format
                  (today_date,stock,CurPrice,Value[stock],round(Tvalue,0),round(MyCash,0))) #卖出处理：交易信息输出-------------

    if len(MyPosition) < HoldNum :    #查看持仓数量情况，不足则买。
        targetNum = HoldNum - len(MyPosition)
        BuyMoney = MyCash / targetNum
        for i in range(HoldNum):
            if len(MyPosition) == HoldNum :break
            if data.index[i] not in MyPosition.keys():
                name = data.index[i]                                         #待建仓转债名称
                price = round(data.loc[name]['closePriceBond'],3)            #待建仓转债价格
                cb_ration = round(data.loc[data.index[i]]['bondPremRatio'],2)#待建仓转债溢价率
                MyPosition[name] = int(BuyMoney / price / 10)        #买入处理：持仓数量、成本、市值同步增加。 
                Cost[name] = MyPosition[name] * price * 10           #买入处理：持仓数量、成本、市值同步增加。
                Value[name] = MyPosition[name] * price * 10          #买入处理：持仓数量、成本、市值同步增加。                
                MyCash -= Cost[name]* (1+0.0001)                     #买入处理：现金减少。                   统计量更新～～～
                Tvalue += Value[name]                                #买入处理：总持仓市值增加。             统计量更新～～～           
                message ='{} 买入 {}, 价格 {}, 成本 {},持仓市值{} ,现金{}'.format(today_date, name, price, Cost[name],round(Tvalue,0),round(MyCash,0))
                print(message)                                       #买入处理：交易信息输出----------------------
   #仓位符合要求，则持仓统计：总资产，总持仓市值，总现金，总收益率，最高市值，最大回撤……
    T_asset = MyCash + Tvalue                         #更新总资产                                        统计量更新～～～
    ratio = (T_asset-Start_Cash)/Start_Cash*100       #更新总收益率
    if T_asset > HighValue:HighValue = T_asset        #更新最高市值
    if (HighValue - T_asset) / HighValue > Withdraw: #更新最大回撤 
        Withdraw = (HighValue - T_asset) / HighValue 
    message = '{}:最高市值{} , 总市值{} , 持仓市值{} ,现金{} , 收益率{}% , 最大回撤{}%'.format(today_date,round(HighValue,0),round(T_asset,0),round(Tvalue,0),round(MyCash,0),round(ratio,2),round(Withdraw*100,1))
    print(message)
    print('='*20)
    print('\n')

<ipython-input-49-9308ee25c935>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['TriLow'] = data['closePriceBond'] + data['bondPremRatio'] + data['remainSize']     #每天重新计算双低排名


2022-01-04 买入 搜特转债, 价格 110.299, 成本 99269.1,持仓市值99269.0 ,现金900721.0
2022-01-04 买入 嘉美转债, 价格 126.88, 成本 98966.4,持仓市值198236.0 ,现金801745.0
2022-01-04 买入 亚泰转债, 价格 119.33, 成本 99043.9,持仓市值297279.0 ,现金702691.0
2022-01-04 买入 岩土转债, 价格 117.148, 成本 99575.8,持仓市值396855.0 ,现金603105.0
2022-01-04 买入 君禾转债, 价格 130.77, 成本 99385.20000000001,持仓市值496240.0 ,现金503710.0
2022-01-04 买入 孚日转债, 价格 126.024, 成本 99558.96,持仓市值595799.0 ,现金404141.0
2022-01-04 买入 海波转债, 价格 129.614, 成本 99802.78,持仓市值695602.0 ,现金304328.0
2022-01-04 买入 百达转债, 价格 128.96, 成本 99299.2,持仓市值794901.0 ,现金205019.0
2022-01-04 买入 灵康转债, 价格 130.09, 成本 98868.4,持仓市值893770.0 ,现金106141.0
2022-01-04 买入 胜达转债, 价格 135.62, 成本 99002.6,持仓市值992772.0 ,现金7128.0
2022-01-04:最高市值999901.0 , 总市值999901.0 , 持仓市值992772.0 ,现金7128.0 , 收益率-0.01% , 最大回撤0%


2022-01-05:最高市值999901.0 , 总市值999228.0 , 持仓市值992099.0 ,现金7128.0 , 收益率-0.08% , 最大回撤0.1%


2022-01-06:最高市值1009049.0 , 总市值1009049.0 , 持仓市值1001921.0 ,现金7128.0 , 收益率0.9% , 最大回撤0.1%


2022-01-07:最高市值1009049.0 , 总市值1003425.0 , 持仓市值996296.0

2022-04-20:最高市值1018691.0 , 总市值937489.0 , 持仓市值930361.0 ,现金7128.0 , 收益率-6.25% , 最大回撤9.3%


2022-04-21:最高市值1018691.0 , 总市值922925.0 , 持仓市值915797.0 ,现金7128.0 , 收益率-7.71% , 最大回撤9.4%


2022-04-22:最高市值1018691.0 , 总市值916617.0 , 持仓市值909488.0 ,现金7128.0 , 收益率-8.34% , 最大回撤10.0%


2022-04-25:最高市值1018691.0 , 总市值881656.0 , 持仓市值874527.0 ,现金7128.0 , 收益率-11.83% , 最大回撤13.5%


2022-04-26:最高市值1018691.0 , 总市值871032.0 , 持仓市值863904.0 ,现金7128.0 , 收益率-12.9% , 最大回撤14.5%


2022-04-27:最高市值1018691.0 , 总市值877695.0 , 持仓市值870566.0 ,现金7128.0 , 收益率-12.23% , 最大回撤14.5%


2022-04-28:最高市值1018691.0 , 总市值880146.0 , 持仓市值873017.0 ,现金7128.0 , 收益率-11.99% , 最大回撤14.5%


2022-04-29:最高市值1018691.0 , 总市值895938.0 , 持仓市值888810.0 ,现金7128.0 , 收益率-10.41% , 最大回撤14.5%


2022-05-05:最高市值1018691.0 , 总市值908599.0 , 持仓市值901470.0 ,现金7128.0 , 收益率-9.14% , 最大回撤14.5%


2022-05-06:最高市值1018691.0 , 总市值901629.0 , 持仓市值894501.0 ,现金7128.0 , 收益率-9.84% , 最大回撤14.5%


2022-05-09:最高市值1018691.0 , 总市值913987.0 , 持仓市值906858.0 ,现金7128.0 , 收益率-8.6% , 最大回撤14.5%


2022-05-1

2022-08-15:最高市值1033180.0 , 总市值1029245.0 , 持仓市值1028208.0 ,现金1037.0 , 收益率2.92% , 最大回撤14.5%


2022-08-16:最高市值1033761.0 , 总市值1033761.0 , 持仓市值1032724.0 ,现金1037.0 , 收益率3.38% , 最大回撤14.5%


2022-08-17:最高市值1039597.0 , 总市值1039597.0 , 持仓市值1038559.0 ,现金1037.0 , 收益率3.96% , 最大回撤14.5%


2022-08-18:最高市值1039597.0 , 总市值1030781.0 , 持仓市值1029743.0 ,现金1037.0 , 收益率3.08% , 最大回撤14.5%


2022-08-19:最高市值1039597.0 , 总市值1023743.0 , 持仓市值1022705.0 ,现金1037.0 , 收益率2.37% , 最大回撤14.5%


2022-08-22:最高市值1039597.0 , 总市值1025894.0 , 持仓市值1024857.0 ,现金1037.0 , 收益率2.59% , 最大回撤14.5%


2022-08-23:最高市值1039597.0 , 总市值1027736.0 , 持仓市值1026698.0 ,现金1037.0 , 收益率2.77% , 最大回撤14.5%


2022-08-24:最高市值1039597.0 , 总市值1014249.0 , 持仓市值1013212.0 ,现金1037.0 , 收益率1.42% , 最大回撤14.5%


2022-08-25:最高市值1039597.0 , 总市值1001460.0 , 持仓市值1000422.0 ,现金1037.0 , 收益率0.15% , 最大回撤14.5%


2022-08-26:最高市值1039597.0 , 总市值999102.0 , 持仓市值998065.0 ,现金1037.0 , 收益率-0.09% , 最大回撤14.5%


2022-08-29:最高市值1039597.0 , 总市值1001004.0 , 持仓市值999967.0 ,现金1037.0 , 收益率0.1% , 最大回撤14.5%


20